In [1]:
#i have used cloud_vision for the text detection and recognition task. 
#preprocessing: preprocessed the raw imges to remove any kind of shadows.
#i have tried multiple ways of preprocessing like morphological transformations, edge detection, finding patches
#in images, thresholding, normalizing, segmenting ROI but to generalize the process for any kind of images
#i have choosed to only remove patches from images
#to avoid multiple requests to the cloud i have made all the requests and stored the json responses into files
#parsed the text from json files as per the code guidelines
#i have used regex to find regestration, chassis numbers
#used some keyword like NAME, REG, ENO just to nevigate to the specific line
#extrating names was quite triky as names i was getting are not of required person(they are of gov organization or vehicle brand)
#to avoid this i have tried 'spacy' library which describes each word with tags like PERSON, ORGANIZATION, etc.
#but i droped idea due to decreasing in no of detections
#to avoid spell mistakes i have also tried 'pyspellchecker' but it can't be used on names of persons
#i have also tried 'pydictionary' to avoid spell correction but some names like 'AMAR NATH' in the given dataset,
#word AMAR actually exists in dicionary so discarding words present in dictionary is getting end up with less accuracy 
#hits finally i end up with 37/46 corrects hits in names
#blank rows in output shows no such attribute was present in image


In [2]:
from PIL import Image
import cv2
import os
import numpy as np
import re
import json
import pandas as pd

In [3]:
li=os.listdir('RC/')  

for im in li:
    img=cv2.imread('RC/'+im,0)
    dilated_img = cv2.dilate(img, np.ones((101,101), np.uint8))   #dilation 
    bg_img = cv2.medianBlur(dilated_img, 5)                       #removing small unwanted patches
    diff_img = 255 - cv2.absdiff(img, bg_img)
    norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    cv2.imwrite('pp/'+im,norm_img)


In [4]:
#json parsing

li=os.listdir('RC/')
lii=os.listdir('JSON/')
texts=[]
for j,i in zip(lii,li):
    img=cv2.imread('RC/'+i)
    text=""
    f = open('JSON/'+j,)  
    response = json.load(f)
    document = response['fullTextAnnotation']
    x1=0
    y1=0
    x2=img.shape[1]
    y2=img.shape[0]
    for page in document['pages']:
        for block in page['blocks']:
            for paragraph in block['paragraphs']:
                for word in paragraph['words']:
                    for symbol in word['symbols']:
                        try:
                            min_x=min(symbol['boundingBox']['vertices'][0]['x'],symbol['boundingBox']['vertices'][1]['x'],symbol['boundingBox']['vertices'][2]['x'],symbol['boundingBox']['vertices'][3]['x'])
                            max_x=max(symbol['boundingBox']['vertices'][0]['x'],symbol['boundingBox']['vertices'][1]['x'],symbol['boundingBox']['vertices'][2]['x'],symbol['boundingBox']['vertices'][3]['x'])
                            min_y=min(symbol['boundingBox']['vertices'][0]['y'],symbol['boundingBox']['vertices'][1]['y'],symbol['boundingBox']['vertices'][2]['y'],symbol['boundingBox']['vertices'][3]['y'])
                            max_y=max(symbol['boundingBox']['vertices'][0]['y'],symbol['boundingBox']['vertices'][1]['y'],symbol['boundingBox']['vertices'][2]['y'],symbol['boundingBox']['vertices'][3]['y'])
                            if(min_x >= x1 and max_x <= x2 and min_y >= y1 and max_y <= y2):
                                text+=symbol['text']
                                if 'property' in symbol and 'detectedBreak' in symbol['property'] :
                                    if(symbol['property']['detectedBreak']['type']=='SPACE') :
                                        text+=' '
                                    if(symbol['property']['detectedBreak']['type']=='SURE_SPACE'):
                                        text+='\t'
                                    if(symbol['property']['detectedBreak']['type']=='LINE_BREAK' or symbol['property']['detectedBreak']['type']=='EOL_SURE_SPACE'):
                                        text+='\n'
                        except:
                            continue
    texts.append(text)

In [5]:
regno=[]
chno=[]
names=[]
engno=[]
regdates=[]
mfgdates=[]
for text in texts:
    text1=text.upper().replace('NEW','').replace(' ','')
    regx=re.search(r'\b[0-9A-Z]{8,9}[0-9]\b',text1)
    if regx:
        regno.append(regx.group(0))
    else:
        regno.append('')
        
        
    regx=re.search(r'[0-9A-Z]{14,16}[0-9]',text1)
    if regx:
        if text1[regx.start():regx.start()+2]=='NO':
            chno.append(text1[regx.start()+2:regx.end()+2])
        else: 
            chno.append(regx.group(0))
    else:
        chno.append('')
        
     
    text2=text.upper()
    regx=text2.find('NAME')
    regx=re.search(r"[A-Za-z .]+",text2[regx+5:regx+30])
    if regx:
        names.append(regx.group(0).strip())
    else:
        names.append('')
        
        
    text3=text.upper()
    regx=text3.find('E NO')
    offset=4
    if regx==-1:
        regx=text3.find('ENGINE NO')
        offset=9
    if regx==-1:
        regx=text3.find('ENGINE')
        offset=6
    if regx==-1:
        engno.append('')
    else:
        regx=re.search(r"[A-Za-z0-9][^ ]+",text3[regx+offset:])
        if regx :
            engno.append(regx.group(0).split('\n')[0].strip())
        else:
            engno.append('')
        
        
        
    text4=text.upper()
    regx=text4.find('REG')
    offset=3
    regx=re.search(r"(\d+/\d+/\d+|\d+/\d{4})",text4[regx+offset:])
    if regx:
        txt=regx.group(0)
        if int(txt[-4:])<=2020:
            regdates.append(txt.split('\n')[0].strip())
        else:
            regdates.append('')
    else:
        regdates.append('')
    
    text5=text.upper()
    regx=text5.find('MFG')
    offset=3
    regx=re.search(r"(\d+/\d+/\d+|\d+/\d{4})",text5[regx+offset:])
    if regx:
        txt=regx.group(0)
        if int(txt[-4:])<=2020:
            mfgdates.append(txt.split('\n')[0].strip())
        else:
            mfgdates.append('')
    else:
        mfgdates.append('')
    

In [6]:
data={'image':li,
      'Name':names,
      'Registration No.':regno,
      'Chassis No.':chno,
      'Engine No.':engno,
      'Reg. DT':regdates,
      'Mfg. DT':mfgdates}

df=pd.DataFrame(data)
df.to_csv('output.csv')

In [7]:
df

,image,Name,Registration No.,Chassis No.,Engine No.,Reg. DT,Mfg. DT
0,txt_mudit_b11_11597.jpg,SRISHTI NAYAR,DL9CAC6215,MA3FHEB1S00358580,D13A0338461,24/12/2012,12/2012
1,txt_mudit_b11_11599.jpg,SRISHTI NAYAR,DL9CAC6215,MA3FHEB1S00358580,D13A0338461,24/12/2012,12/2012
2,txt_mudit_b11_12.jpg,ANOOP SURESH DHAWALE,DL2CAT9109,MABETDE1S00218363,7567094,21/07/2015,07/2015
3,txt_mudit_b11_1328.jpg,SUBE SINGH,HRO6P5988,ENGINEF8DN3321864,F8DN3321864,9/2006,9/2006
4,txt_mudit_b11_1330.jpg,RANBEER,HRO6S8814,MA3EYD81801277497,F8DN1266647,11/2008,11/2008
5,txt_mudit_b11_1332.jpg,AMAR NATH,HR49D0002,MA3FJEB1S00404062,D13A2235550,9/2013,9/2013
6,txt_mudit_b11_1334.jpg,MR HARISH KUMAR,HROGAH1768,MAJELMG1800384268,K14BN,4/2016,4/2016
7,txt_mudit_b11_1337.jpg,PAWAN KUMAR,HR14K6035,ONTHANDYEAROFMIG7,D13A%2265667,7/2013,7/2013
8,txt_mudit_b11_1339.jpg,MR VINAY KUMAR HANDA,HROBAE8243,CHASSISNOMAZEJKD1,K12MN,1/2015,1/2015
9,txt_mudit_b11_1343.jpg,MR. JITENDER,HROGAK2102,MA3FHEB1SO0B52684,D13A2899227,9/2016,9/2016
